In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('pa2train.txt',header=None,delimiter=' ')
train = train.drop(23, axis = 1)
test = pd.read_csv('pa2test.txt',header=None,delimiter=' ')
test = test.drop(23, axis = 1)
validate = pd.read_csv('pa2validation.txt',header=None,delimiter=' ')
validate = validate.drop(23, axis = 1)
features = pd.read_csv('pa2features.txt', header = None)

In [3]:
# initialize table with feature 1 - 22, label the 'target' columns
train.columns = [i + 1 for i in range(22)] + ['target']
test.columns = [i + 1 for i in range(22)] + ['target']
validate.columns = [i + 1 for i in range(22)] + ['target']

In [4]:
### Splitting data into feature DataFrame X and target Series y
X_train = train.iloc[:, :-1]
y_train = train.iloc[:, -1]
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]
X_validate = validate.iloc[:, :-1]
y_validate = validate.iloc[:, -1]

In [5]:
X_train.shape

(2000, 22)

In [7]:
y_train

0       1.0
1       1.0
2       1.0
3       1.0
4       0.0
5       0.0
6       1.0
7       1.0
8       1.0
9       1.0
10      0.0
11      1.0
12      0.0
13      1.0
14      1.0
15      1.0
16      0.0
17      0.0
18      0.0
19      1.0
20      0.0
21      1.0
22      1.0
23      1.0
24      0.0
25      0.0
26      1.0
27      0.0
28      1.0
29      0.0
       ... 
1970    0.0
1971    0.0
1972    1.0
1973    0.0
1974    0.0
1975    0.0
1976    0.0
1977    0.0
1978    1.0
1979    1.0
1980    1.0
1981    0.0
1982    1.0
1983    1.0
1984    0.0
1985    1.0
1986    1.0
1987    1.0
1988    1.0
1989    0.0
1990    1.0
1991    1.0
1992    0.0
1993    0.0
1994    0.0
1995    0.0
1996    0.0
1997    1.0
1998    0.0
1999    1.0
Name: target, Length: 2000, dtype: float64

In [5]:
### Classifier ###
class ID3():
    '''Initialize ID3 Class'''
    class Node():
        '''Tree Node'''
        def __init__(self, feature = None, threshold= None, samples = None, label = None):
            self.left = None
            self.right = None
            self.feature = feature
            self.threshold = threshold
            self.samples = samples
            self.label = label
            self.isleft = None
            self.parent = None
        def set_left(self, other): 
            '''Set left child'''
            self.left = other
            other.isleft = True
            other.parent = self
        def set_right(self, other):
            '''Set right child'''
            self.right = other
            other.isleft = False
            other.parent = self
        def is_leaf(self):
            '''Determine if the Node is leaf'''
            return self.left == None and self.right == None
        def print_info(self):
            '''Print info of the Node'''
            if self.is_leaf():
                print(self.label)
            else:
                print(self.feature, self.threshold, self.samples)
    def _H(self, S):
        '''Helper function of Getting Entropy of S'''
        p = np.unique(S,return_counts=True)[1]/len(S)
        entropy = np.sum(-p*(np.log2(p)))
        return entropy

    def _IG(self,train,target,threshold):
        '''Helper function of Getting Information Gain of training set over target column with certain threshold'''
        t1 = target.loc[train>=threshold]
        t2 = target.loc[train<threshold]

        p = len(t1)/len(train)
        return self._H(target)-p*self._H(t1)-(1-p)*self._H(t2)
    def __init__(self):
        '''Constructor of ID3'''
        self.tree = None #A decision tree in .tree
    
    def _ID3_func(self,X,y,_mode,_prev_mode):
        '''Helper function to build the decision tree'''
        ### base cases ###:
        labels = y.unique()
        # When the dataset is pure
        if len(labels)==1:
            node = self.Node(label=labels[0])
            return node
        # When the dataset is empty Node as the mode overall distribution
        if len(X) == 0:
            return self.Node(label = _mode)
        if len(X.columns) == 0:
        # When the dataset's feature is empty, label the Node as previous mode
            return self.Node(label = _prev_mode)
        
        ### Calculate the entropy & best feature & best threshold###
        gain = []
        threshold = []
        for feature in X.columns:
            x = X.loc[:,feature]
            temp = np.unique(x)
            candidates = np.diff(temp)/2+temp[:-1] 
            ig = [self._IG(x,y,c) for c in candidates]
            if len(ig) == 0:
                gain.append(-np.inf)
                threshold.append(-np.inf)
                continue
            else:
                gain.append(np.amax(ig))
                threshold.append(candidates[np.argmax(ig)])
        ix = np.argmax(gain)
        best_feature = X.columns[ix]
        best_threshold = threshold[ix]
        ### Construct the Tree Node ###
        tree = self.Node(feature = best_feature,threshold = best_threshold, samples = len(X))
        # Splitting the dataset into two piece
        mask = X[best_feature]<best_threshold
        X_left = X.loc[mask]
        y_left = y.loc[mask]
        X_right = X.loc[~mask]
        y_right = y.loc[~mask]
        ### Create left node and right node by recursion ###
        left_node = self._ID3_func(X_left, y_left, _mode, y.unique()[0])
        right_node = self._ID3_func(X_right, y_right, _mode, y.unique()[0])
        tree.set_left(left_node)
        tree.set_right(right_node)
        return tree
    def fit(self, X, y):
        '''To fit the decision tree to classifier'''
        _mode = y.unique()[0]
        self.tree = self._ID3_func(X, y, _mode, _mode)
    
    def _predict(self, Xi, curr = None):
        '''Helper function for Predict Xi'''
        if curr == None:
            # If it curr is None, start from root
            curr = self.tree
        if curr.is_leaf():
            # Base Case, if current node is leaf, return the label
            to_return = curr.label
            return to_return
        # Get splitting decision
        feature, threshold = curr.feature, curr.threshold
        # Classify the Xi into next node
        if Xi[feature] < threshold:
            curr = curr.left
            return self._predict(Xi, curr)
        else:
            curr = curr.right
            return self._predict(Xi, curr)
    def predict(self, X):
        '''funtion to predict X dataset.'''
        return pd.Series([self._predict(X.iloc[Xi,:]) for Xi in range(X.shape[0])], index = X.index)
    def score(self, X, y):
        '''Get the accuracy of prediction'''
        return (self.predict(X) == y).mean()

            
    def _get_sample(self, X, Node):
        ''' Get the samples in this Node by reverse the tree'''
        if Node == self.tree:
            # Base Case Node is root, return sample
            return X
        # Traverse back to parent Node with parent's decision
        elif Node.isleft == True:
            feature, threshold = Node.parent.feature, Node.parent.threshold
            X = X[X[feature] < threshold]
            return self._get_sample(X, Node.parent)
        elif Node.isleft == False:
            feature, threshold = Node.parent.feature, Node.parent.threshold
            X = X[X[feature] >= threshold]
            return self._get_sample(X, Node.parent)
    def _bfs(self):
        '''traverse the decision tree by bfs'''
        thislevel = [self.tree]
        to_return = thislevel
        while thislevel:
            nextlevel = list()
            for n in thislevel:
                if n.left: 
                    nextlevel.append(n.left)
                if n.right: 
                    nextlevel.append(n.right)

            thislevel = nextlevel
            to_return += nextlevel
        return to_return #return as a list
    
    def prune(self, X, y, N):
        nodes = self._bfs()
        # Looping over the bfs of each node by bfs
        for node in nodes:
            # Get samples in the node
            X_sample = self._get_sample(X, node)
            y_sample = y[y.index.isin(X_sample.index)]
            # Calculating err of classifier
            err = 1 - self.score(X_sample, y_sample)
            # Get the most freq label of the prediction
            mode_i = self.predict(X_sample).value_counts().idxmax()
            # Calculate Prune decision
            prune_pred = pd.Series([mode_i for Xi in range(len(X_sample))], index = X_sample.index)
            err_hat = 1 - (prune_pred == y_sample).mean()
            # Replace Node if err_hat is smaller or equal
            if err_hat <= err:
                new_node = self.Node(label = mode_i)
                if node.isleft == True:
                    node.parent.set_left(new_node)
                elif node.isleft == False:
                    node.parent.set_right(new_node)
                else:
                    node = new_node
                N -= 1
            # Stopping
            if N == 0:
                break


    


In [6]:
clf = ID3()

In [7]:
clf.fit(X_train, y_train)

In [8]:
X_train

,1,2,3,4,5,6,7,8,9,10,...,13,14,15,16,17,18,19,20,21,22
0,30000.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,32415.0,30324.0,31124.0,25502.0,2506.0,2430.0,1300.0,1000.0,34436.0,950.0
1,160000.0,1.0,1.0,2.0,4.0,3.0,2.0,2.0,3.0,2.0,...,99911.0,103928.0,101540.0,99587.0,0.0,5500.0,6700.0,0.0,27.0,2800.0
2,190000.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,...,38051.0,38827.0,39488.0,40208.0,1900.0,1800.0,1700.0,1600.0,1500.0,1600.0
3,140000.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1446.0,2880.0,1914.0,968.0,2000.0,1500.0,3000.0,1914.0,0.0,3287.0
4,200000.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9427.0,2040.0,16338.0,2076.0,8290.0,9427.0,2240.0,16000.0,2076.0,833.0
5,140000.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,117263.0,77496.0,79815.0,77911.0,5200.0,5006.0,5000.0,5000.0,3000.0,5000.0
6,130000.0,1.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,...,126862.0,97925.0,100484.0,54125.0,0.0,5000.0,3500.0,4055.0,10000.0,50000.0
7,110000.0,2.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,...,44126.0,43483.0,43435.0,41010.0,0.0,3000.0,2000.0,2000.0,2000.0,3600.0
8,200000.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,479.0,0.0,0.0,0.0,0.0,0.0
9,200000.0,2.0,1.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,...,823.0,823.0,-658.0,1266.0,2250.0,0.0,0.0,0.0,1924.0,0.0


In [8]:
def print_tree(Node, height):
    '''Helper funtion to print tree'''
    def print_given_level(root, level):
        if root.is_leaf():
            print(root.label)
            return
        elif level == 1:
            print((root.feature, root.threshold, root.samples), end = '      ')
            return
        else:
            print_given_level(root.left, level - 1)
            print_given_level(root.right, level - 1)
        
    
    for i in range(height):  
        print((height - i) * '               ', end = '')
        print_given_level(Node, i+1)
        print()
    

## Question 1

In [9]:
print_tree(clf.tree, 3)

                                             (5, 0.5, 2000)      
                              (1, 415000.0, 1319)      (5, 1.5, 681)      
               (17, 2506.5, 1284)      (21, 208.0, 35)      (20, 584.5, 292)      (21, 2006.0, 389)      


Above are the first 3 depth Nodes of the decision tree without pruning, For example,the printing message of one node: (5, 0.5, 2000) means the feature 5 with < 0.5 on left >= 0.5 on right, and 2000 samples in this node

## Question 2

In [10]:
output = pd.DataFrame(
        {
            'Training Error': 1 - clf.score(X_train, y_train),
            'Validation Error' : 1 - clf.score(X_validate, y_validate),
            'Test Error': 1 - clf.score(X_test, y_test)
        }, index = ['without pruning']
)

In [11]:
output

,Training Error,Validation Error,Test Error
without pruning,0.0,0.179,0.173


## Question 3

In [12]:
clf.prune(X_validate, y_validate, 1)

In [13]:
output = pd.DataFrame(
        {
            'Training Error': 1 - clf.score(X_train, y_train),
            'Validation Error' : 1 - clf.score(X_validate, y_validate),
            'Test Error': 1 - clf.score(X_test, y_test)
        }, index = pd.Series([1],name = 'Prune')
)

In [14]:
clf = ID3()
clf.fit(X_train, y_train)
clf.prune(X_validate, y_validate, 2)

In [15]:
output = output.append(pd.DataFrame(
        {
            'Training Error': 1 - clf.score(X_train, y_train),
            'Validation Error' : 1 - clf.score(X_validate, y_validate),
            'Test Error': 1 - clf.score(X_test, y_test)
        }, index = pd.Series([2],name = 'Prune')
))

In [16]:
display(output) #Display the error of prune

,Training Error,Validation Error,Test Error
Prune,,,
1,0.0845,0.122,0.117
2,0.1050,0.107,0.103


In [17]:
# see the which node is pruned
print(clf.tree.left.label, clf.tree.right.label)

0.0 1.0


In [18]:
#we observe that the left child and right child of root have been pruned.

## Question 4

According to the tree in question 1, and in previous question, we found that we pruned the left and right node from the root, it still has low validation error and test error. Then the fifth feature is the most salient feature. And feature 1, 17, 20 21 are also relevent.

In [19]:
print(features.iloc[4][0])

PAYMENT_DELAY_SEPTEMBER


In [20]:
print(features.iloc[[0,16,19,20]][0].tolist())

['LIMIT_BAL', 'PAY_AMT1', 'PAY_AMT4', 'PAY_AMT5']


As the cell above shown, the fifth feature is 'PAYMENT_DELAY_SEPTEMBER', which is the most salient feature.

'LIMIT_BAL', 'PAY_AMT1', 'PAY_AMT4', 'PAY_AMT5' are also relevent